## Read CVE-fixes database and analyze it

In [6]:
import pandas as pd 
import sqlite3 

cvefixes_db = '/Users/guru/Simula_secureIT/experiment/CVEfixes_v3.db'

# function to read the database and return a dataframe
def read_db(db):
    # connect to the database
    conn = sqlite3.connect(db)
    # read the database into a dataframe
    df = pd.read_sql_query("SELECT * FROM method_change", conn)
    # close the connection
    conn.close()
    # return the dataframe
    return df

# get all CVEs from the DB
df = read_db(cvefixes_db)

In [8]:
df_fun  = df[['code','before_change']]
df_fun = df_fun.drop_duplicates()
df_fun = df_fun.dropna()
df_fun = df_fun.reset_index(drop=True)
df_fun

,code,before_change
0,expand_dynamic_string_token (struct link_map *...,True
1,"_dl_dst_substitute (struct link_map *l, const ...",True
2,"_dl_dst_count (const char *name, int is_path)\...",True
3,expand_dynamic_string_token (struct link_map *...,False
4,"_dl_dst_substitute (struct link_map *l, const ...",False
...,...,...
152281,SecureElementStatus_t SecureElementProcessJoin...,False
152282,"queue.onData = function(paths, end) {\n v...",True
152283,"Unix.files = function(paths, end) {\n var sel...",True
152284,"queue.onData = function(paths, end) {\n v...",False


In [120]:
def get_patch(df_min):
    # sort df_min based on signature and then before_change
    df_min = df_min.sort_values(by=['signature','before_change'])
    df_min = df_min.reset_index(drop=True)
    df_patch = pd.DataFrame(columns=['vul','fix'])

    for num in range(0, len(df_min)-1):
        if df_min.signature[num]==df_min.signature[num+1]:
            # print(df_min.before_change[num] + ' ---> ' + df_min.signature[num])
            # print(df_min.before_change[num+1] + '  ---> ' + df_min.signature[num+1])
            vul = ''
            fix = ''
            if df_min.before_change[num]=='False':
                fix=  df_min.code[num]
            if df_min.before_change[num+1]=='True':
                vul = df_min.code[num+1]
            
            if vul!='' or fix!='':
                patch= {'vul': vul, 'fix': fix}
                df_patch = pd.concat([df_patch, pd.DataFrame([patch])], ignore_index=True)
        # print('-----------------------------')

    return df_patch

In [123]:
file_id = list(df.file_change_id.unique())
df_patch_all = pd.DataFrame(columns=['vul','fix'])

for id in file_id:
    df_min = df[df.file_change_id==id]
    df_patch = get_patch(df_min)
    df_patch_all = pd.concat([df_patch_all, df_patch], ignore_index=True)

df_patch_all

,vul,fix
0,"_dl_dst_count (const char *name, int is_path)\...","_dl_dst_count (const char *name, int is_path)\..."
1,"_dl_dst_substitute (struct link_map *l, const ...","_dl_dst_substitute (struct link_map *l, const ..."
2,expand_dynamic_string_token (struct link_map *...,expand_dynamic_string_token (struct link_map *...
3,"vsyslog(pri, fmt, ap)\n\tint pri;\n\tregister ...","vsyslog(pri, fmt, ap)\n\tint pri;\n\tregister ..."
4,init_syntax_once ()\n{\n register int c;\n ...,init_syntax_once ()\n{\n register int c;\n ...
...,...,...
19360,def load_graphviz_file(ini_f):\n '''\n L...,def load_graphviz_file(ini_f):\n '''\n L...
19361,"def save_graphviz_file(ini_f, attr_d):\n ''...","def save_graphviz_file(ini_f, attr_d):\n ''..."
19362,static void ProcessRadioRxDone( void )\r\n{\r\...,static void ProcessRadioRxDone( void )\r\n{\r\...
19363,"Unix.files = function(paths, end) {\n var sel...","Unix.files = function(paths, end) {\n var sel..."


In [129]:
df_patch_all.drop_duplicates().to_csv('data/cvefixes-patches.csv')